# Deploy `BAAI/bge-large-en-v1.5` Text Embedding Model (1024 Dimension) to Amazon SageMaker

In this notebook, we demonstrate, how we can package and deploy `BAAI/bge-large-en-v1.5` embedding model with **1024** dimensions.

`bge` is short for BAAI general embedding.

*NOTE*: If you need to search the long relevant passages to a short query (s2p retrieval task), you need to add the instruction to the query; in other cases, no instruction is needed, just use the original query directly. In all cases, no instruction need to be added to passages.


**NOTE:** bge model sizes and dimension
- `BAAI/bge-large-en-v1.5`: **~1.34GB** (Dimensions: 1024)
- `BAAI/bge-base-en-v1.5`: **~438MB** (Dimensions: 768)

## References
- **Model Card:** <https://huggingface.co/BAAI/bge-large-en-v1.5#using-huggingface-transformers>
- <https://github.com/FlagOpen/FlagEmbedding>

## Inference script

Refer to [./models/bi-encoders/bge-large-en-v15/code/inference.py](./models/bi-encoders/bge-large-en-v15/code/inference.py) for implementation details.

In [ ]:
# !pip install -Uq boto3 sagemaker rich watermark ipywidgets
# %load_ext rich
# %load_ext watermark
# %watermark -p boto3,sagemaker,ipywidgets
# %watermark -m -t -v

boto3     : 1.28.21
sagemaker : 2.177.2.dev0
ipywidgets: 7.8.1

Python implementation: CPython
Python version       : 3.9.16
IPython version      : 8.12.0

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 10
Architecture: 64bit



In [ ]:
import shutil
import sys
from pathlib import Path

from huggingface_hub import snapshot_download
from rich import print
from sagemaker import get_execution_role
from sagemaker.deserializers import JSONDeserializer
from sagemaker.s3 import S3Uploader, s3_path_join
from sagemaker.serializers import JSONSerializer
from sagemaker.session import Session

sys.path.append("./utils")
from utils import sm_utils, utils

In [ ]:
session = Session()
bucket = session.default_bucket()
role = get_execution_role()
region = session.boto_region_name

HF_MODEL_ID = "BAAI/bge-large-en-v1.5"
model_base_name = HF_MODEL_ID.split("/")[-1].replace(".", "")
model_folder = Path(f"./models/bi-encoders/{model_base_name}")
model_archive_path = model_folder.joinpath("model.tar.gz")
s3_baseuri = s3_path_join(
    f"s3://{bucket}/models", f"txt-embedding-models/{model_base_name}"
)

print(f"Region: [i]{region}[/i]")
print(f"bucket name: {s3_baseuri}")
print(f"Model dir: {model_folder}")

Region: us-west-2

bucket name: s3://sagemaker-us-west-2-726793866085/models/txt-embedding-models/bge-large-en-v15

Model dir: models/bi-encoders/bge-large-en-v15

In [ ]:
model_bin = model_folder.joinpath("pytorch_model.bin")

if not model_bin.exists():
    print("Downloading model ...")
    snapshot_download(
        repo_id=HF_MODEL_ID,
        local_dir=model_folder,
        local_dir_use_symlinks=False,
        allow_patterns=["1_Pooling", "*.txt", "*.json", "*.bin"],
    )
else:
    print(f"Model already downloaded. {model_folder}")

### Create Model

- Compress model artifacts to `model.tar.gz`
- Upload model to S3
- Create Model object


In [ ]:
utils.clear_ipynb_dirs(model_folder)  # remove any .ipynb_checkpoints, __pycache__
model_archive_path = model_folder.joinpath("model.tar.gz")
if model_archive_path.exists():
    print(f"Deleting existing model: {model_archive_path}")
    model_archive_path.unlink(missing_ok=True)

print(f"Creating archive with base_dir={model_folder}")
model_archive_path = shutil.make_archive(
    format="gztar",  # tar.gz format
    base_name=model_folder.name,  # will create model.tar.gz
    root_dir=model_folder,  # dir to chdir into before archiving
)

Creating archive with base_dir=models/bi-encoders/bge-large-en-v15

In [ ]:
# Verify contents of the model archive.
!tar tvf $model_archive_path

drwxr-xr-x  0 pchamart staff       0 Nov  1 15:01 ./
drwxr-xr-x  0 pchamart staff       0 Nov  1 14:59 ./1_Pooling/
-rw-r--r--  0 pchamart staff     191 Nov  1 14:59 ./1_Pooling/config.json
drwxr-xr-x  0 pchamart staff       0 Nov  1 15:02 ./code/
-rw-r--r--  0 pchamart staff    2251 Nov  1 10:57 ./code/inference.py
-rw-r--r--  0 pchamart staff     779 Nov  1 14:59 ./config.json
-rw-r--r--  0 pchamart staff     124 Nov  1 14:59 ./config_sentence_transformers.json
-rw-r--r--  0 pchamart staff     349 Nov  1 14:59 ./modules.json
-rw-r--r--  0 pchamart staff 1340698349 Nov  1 15:01 ./pytorch_model.bin
-rw-r--r--  0 pchamart staff         52 Nov  1 14:59 ./sentence_bert_config.json
-rw-r--r--  0 pchamart staff        125 Nov  1 14:59 ./special_tokens_map.json
-rw-r--r--  0 pchamart staff     711396 Nov  1 15:00 ./tokenizer.json
-rw-r--r--  0 pchamart staff        366 Nov  1 15:00 ./tokenizer_config.json
-rw-r--r--  0 pchamart staff     231508 Nov  1 15:00 ./vocab.txt


### Upload archive to S3

In [ ]:
print(f"Uploading model from {model_archive_path} to \n{s3_baseuri} ...")
model_data_url = S3Uploader.upload(
    local_path=str(model_archive_path),
    desired_s3_uri=s3_baseuri,
    sagemaker_session=session,
)
print(f"Model Data URL: {model_data_url}")

Uploading model from /Users/pchamart/dev/ai-ml/nlp/deploy-embedding-models-aws/bge-large-en-v15.tar.gz to 
s3://sagemaker-us-west-2-726793866085/models/txt-embedding-models/bge-large-en-v15 ...

Model Data URL: 
s3://sagemaker-us-west-2-726793866085/models/txt-embedding-models/bge-large-en-v15/bge-large-en-v15.tar.gz

Create HuggingFaceModel with model data and custom `inference.py` script

https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#hugging-face-model


### Deploy to real-time endpoint

Serveless endpoint can only host models with total image size + model size <= 10GB

The HuggingFace Transformers Model container + BGE Basemodel exceeds 10GB So, we deploy to real-time endpoint here.

Helper functions to create and deploy huggingface model is under [utils (sm_utils)](./utils/sm_utils.py) module.

In [ ]:
instance_type = "ml.c5.4xlarge"
suffix = utils.get_suffix()  # returns a uui4-datetime formatted string
model_name = f"{model_base_name}-{suffix}"
env = {"HF_TASK": "feature-extraction"}  # HF_TASK is required for HF models

# function to create and deploy model to real-time endpoint
predictor = sm_utils.create_deploy_huggingface_model(
    model_name=model_name,
    model_s3uri=model_data_url,
    role=role,
    instance_type=instance_type,
    env=env,
)

Creating model: bge-large-en-v15-6ec44-01Nov2023

Deploying to endpoint:bge-large-en-v15-6ec44-01Nov2023 with 

env={'HF_TASK': 'feature-extraction'}

Deploying bge-large-en-v15-6ec44-01Nov2023 to ml.c5.4xlarge real-time endpoint

### Wait for endpoint to be `InService` state

In [ ]:
sm_utils.get_endpoint_status(endpoint_name=model_name)

Endpoint: bge-large-en-v15-6ec44-01Nov2023 | Status: Creating

Endpoint: bge-large-en-v15-6ec44-01Nov2023 | Status: InService ✅

### Inference

Refer to [models/bi-encoders/bge-large-en-v15/code/inference.py](./models/bi-encoders/bge-large-en-v15/code/inference.py) for complete implementation.

**Model Card:** <https://huggingface.co/BAAI/bge-large-en-v1.5#using-huggingface-transformers>

```python
def generate_embeddings(texts, model, tokenizer, normalize=True):
    """
    Generate embeddings for a list of texts using a pre-trained model.

    Args:
        texts (List[str]): List of texts to calculate embeddings for.
        model (AutoModel): Pre-trained model.
        tokenizer (AutoTokenizer): Tokenizer corresponding to the pre-trained model.
        normalize (bool, optional): Whether to normalize the embeddings. Defaults to True.

    Returns:
        Tensor: Tensor containing the embeddings for the texts.
    """

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Tokenize the texts
    encoded_input = tokenizer(
        texts, max_length=512, padding=True, truncation=True, return_tensors="pt"
    )

    encoded_input = encoded_input.to(device)

    # Get the embeddings for the texts
    with torch.no_grad():
        model_output = model(**encoded_input)

        # Perform pooling. In this case, cls pooling.
        sentence_embeddings = model_output[0][:, 0]


    # Normalize embeddings if required
    if normalize:
        sentence_embeddings = F.normalize(text_embeddings, p=2, dim=1)

    # convert to numpy array
    sentence_embeddings = sentence_embeddings.cpu().numpy()
    ret_value = sentence_embeddings.tolist()

    return ret_value

...

```



### Invoke Endpoint

Before we invoke we attach `JSONSerializer` and `JSONDeserializer` to the predictor object.

In [ ]:
predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

In [ ]:
sentences = ["That is a happy person", "That is a very happy person"]

embeddings = predictor.predict(sentences)

print(f"Embedding dimensions: {len(embeddings[0])}")
print(embeddings[0])

Embedding dimensions: 1024

[
    -0.016109079122543335,
    0.03207748383283615,
    -0.006008266471326351,
    0.011022028513252735,
    -0.026171868667006493,
    -0.03405395522713661,
    0.022993948310613632,
    0.044599052518606186,
    0.04573546722531319,
    -0.0219249427318573,
    0.007054363377392292,
    -0.011310209520161152,
    0.0020601372234523296,
    -0.014813169836997986,
    -0.03392254188656807,
    -0.04174813628196716,
    -0.02763097546994686,
    -0.03948516026139259,
    -0.04474719986319542,
    0.010473474860191345,
    -0.004775356035679579,
    0.007269349414855242,
    -0.06708170473575592,
    -0.013792131096124649,
    0.02481500245630741,
    0.061570800840854645,
    0.010901080444455147,
    0.03207390010356903,
    0.07409502565860748,
    0.05470365285873413,
    -0.02660243958234787,
    0.013931735418736935,
    0.030067387968301773,
    -0.058148615062236786,
    0.018813475966453552,
    -0.023004252463579178,
    0.012345964089035988,
    -0.019396759569644928,
    0.045880287885665894,
    -0.026210710406303406,
    0.02171066217124462,
    -0.018904734402894974,
    0.046243056654930115,
    -0.01073622889816761,
    -0.08041059970855713,
    -0.05054293945431709,
    -0.020870186388492584,
    0.003096058964729309,
    0.0646747499704361,
    -0.05141772702336311,
    -0.02059910260140896,
    -0.0004937205230817199,
    0.018228359520435333,
    -0.043200891464948654,
    0.0193802323192358,
    0.005577332340180874,
    0.020074041560292244,
    -0.013819515705108643,
    -0.02879856526851654,
    0.05230817571282387,
    0.015315437689423561,
    0.025128230452537537,
    0.023945951834321022,
    -0.06520134210586548,
    0.02048327773809433,
    0.005608741194009781,
    -0.013549033552408218,
    -0.03103075735270977,
    -0.016325633972883224,
    -0.01307025644928217,
    0.02748251147568226,
    -0.005746235139667988,
    -0.03992410749197006,
    -0.0355960913002491,
    -0.03369734436273575,
    0.053312018513679504,
    -0.025947732850909233,
    0.04118802025914192,
    -0.017428653314709663,
    0.03353913128376007,
    0.01466281060129404,
    0.020584961399435997,
    0.00720206880941987,
    -0.018501531332731247,
    -0.03193997964262962,
    0.013509966433048248,
    0.032830365002155304,
    0.046774789690971375,
    0.004283551126718521,
    -0.03858723118901253,
    0.03651302680373192,
    0.05807937681674957,
    0.022636987268924713,
    -0.005496184807270765,
    0.03921448066830635,
    -0.000221583730308339,
    0.007454192265868187,
    0.040015704929828644,
    0.011086024343967438,
    0.045731108635663986,
    -0.006289680954068899,
    0.0065444777719676495,
    0.03194693475961685,
    0.04843705892562866,
    -0.05837269499897957,
    0.00296341092325747,
    0.016057951375842094,
    -0.008032643236219883,
    -0.035590581595897675,
    -0.023012179881334305,
    -0.008144527673721313,
    -0.028041880577802658,
    0.03222905844449997,
    0.027698513120412827,
    -0.034283772110939026,
    0.029596764594316483,
    0.02315930277109146,
    0.06093486770987511,
    -0.015004702843725681,
    -0.02021666243672371,
    -0.005048814229667187,
    0.011908461339771748,
    0.002702650148421526,
    -0.025761745870113373,
    -0.014703270047903061,
    -0.005277908407151699,
    0.011655250564217567,
    0.05595247447490692,
    -0.010008677840232849,
    -0.015896115452051163,
    0.03585948422551155,
    -0.024843409657478333,
    -0.014787500724196434,
    -0.01762508600950241,
    -0.005627635400742292,
    0.018350224941968918,
    -0.03466559201478958,
    0.05470845848321915,
    0.026238948106765747,
    -0.03468367084860802,
    0.03806057199835777,
    0.023239415138959885,
    0.01631929539144039,
    0.061027642339468,
    -0.03358692303299904,
    0.043102145195007324,
    0.033359263092279434,
    -0.016935164108872414,
    -0.03772587701678276,
    -0.0374426394701004,
    -0.04615600407123566,
    0.01939377561211586,
    -0.028753912076354027,
    0.0158840

## Cleanup

Delete resources after use

In [ ]:
print(f"Deleting model and endpoint: {model_name}")
predictor.delete_model()
predictor.delete_endpoint()

Deleting model and endpoint: bge-large-en-v15-6ec44-01Nov2023